In [2]:
import numpy as np
import os
import json
import matplotlib.pyplot as plt
import utils.feature_extractors as utils
import optuna
import joblib

from sklearn.pipeline import make_pipeline
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from utils.utils import evaluate_classification
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier


In [3]:
config_map = {
    "train_postive_location":"dataset/TR_pos_SPIDER.txt",
    "train_negative_location":"dataset/TR_neg_SPIDER.txt",
    "test_positive_location":"dataset/TS_pos_SPIDER.txt",
    "test_negative_location":"dataset/TS_neg_SPIDER.txt",
    "model_save_location":"./optimized_models",
    "random_seed":9
}

In [4]:
class ProteinFeatureGenerator:
    SELECTED_FEATURES = ["AAC", "DPC", "RScharge", "RSDHP", "RSpolar"]
    
    def __init__(self, positive_data_file: str, negative_data_file: str, feature_type: str = None) -> None:
        super().__init__()

        # Check feature param
        assert feature_type in ProteinFeatureGenerator.SELECTED_FEATURES or feature_type is None
        self.feature_type = feature_type

        # Data manipulation
        self.positive_data_file = positive_data_file
        self.negative_data_file = negative_data_file

        self.positive_data = utils.read_fasta(self.positive_data_file)
        self.negative_data = utils.read_fasta(self.negative_data_file)
        
        self.data = self.positive_data + self.negative_data
        self.targets = np.array([True]*len(self.positive_data) + [False]*len(self.negative_data))

        self.raw_sequences = [x[1] for x in self.data]
        
        
        # Extract features
        print("Extracting AAC Feature ...")
        self.AAC_feature = utils.AAC(self.data)[0]

        print("Extracting DPC Feature ...")
        self.DPC_feature = utils.DPC(self.data, 0)[0]

        print("Extracting RScharge Feature ...")
        self.RScharge_feature = utils.reducedCHARGE(self.data)
        
        print("Extracting RSDHP Feature ...")
        self.RSDHP_feature = utils.reducedDHP(self.data)
        
        print("Extracting RSpolar Feature ...")
        self.RSpolar_feature = utils.reducedPOLAR(self.data)
        
    
    def __len__(self) -> int:
        return len(self.data)

In [5]:
train_data = ProteinFeatureGenerator(positive_data_file=config_map["train_postive_location"],negative_data_file=config_map["train_negative_location"])

Extracting AAC Feature ...
Extracting DPC Feature ...
Extracting RScharge Feature ...
Extracting RSDHP Feature ...
Extracting RSpolar Feature ...


In [6]:
X_data = {
    "AAC":train_data.AAC_feature,
    "DPC":train_data.DPC_feature,
    "RScharge":train_data.RScharge_feature,
    "RSDHP":train_data.RSDHP_feature,
    "RSpolar":train_data.RSpolar_feature,
}

feature_model_map = {
    "AAC":"SVC",
    "DPC":"SVC",
    "RScharge":"RandomForest",
    "RSDHP":"SVC",
    "RSpolar":"ExtraTreesClassifier",
}

data_pipelines = {
    "AAC":make_pipeline(StandardScaler()),
    "DPC":make_pipeline(StandardScaler()),
    "RScharge":make_pipeline(StandardScaler()),
    "RSDHP":make_pipeline(StandardScaler()),
    "RSpolar":make_pipeline(StandardScaler()),
}


In [11]:
class ObjectiveFunctions(object):
    SELECTED_MODELS = ['RandomForest', 'SVC', 'ExtraTreesClassifier']
    
    def __init__(self, model_name: str) -> None:
        # Check model name
        assert model_name in ObjectiveFunctions.SELECTED_MODELS, "Invalid Model Name."
        self.model_name = model_name
    
    @staticmethod
    def obj_func_svc(trial: optuna.trial) -> SVC:
        c = trial.suggest_float('C', 1e-10, 1e10, log=True)
        kernel = 'rbf'
        gamma = 'auto'
        
        classifier = SVC(
            C=c, 
            kernel=kernel,
            class_weight={1: 0.482, 0: 0.518},
            gamma=gamma,
        )
        
        return classifier
    
    @staticmethod
    def obj_func_random_forest(trial: optuna.trial) -> RandomForestClassifier:
        n_estimators = trial.suggest_int('n_estimators', 75, 300)
        max_depth = trial.suggest_int('max_depth', 1, 64)
        criterion = trial.suggest_categorical("criterion", ['gini', 'entropy'])
        
        classifier = RandomForestClassifier(
            n_estimators=n_estimators, 
            max_depth=max_depth,
            criterion=criterion,
            class_weight={1: 0.482, 0: 0.518},
        )
        
        return classifier
    
    
    @staticmethod
    def obj_func_extra_tree(trial: optuna.trial) -> ExtraTreesClassifier:
        n_estimators = trial.suggest_int('n_estimators', 75, 300)
        max_depth = trial.suggest_int('max_depth', 1, 64)
        criterion = trial.suggest_categorical("criterion",['gini', 'entropy', 'log_loss'])
        
        classifier = ExtraTreesClassifier(
            n_estimators=n_estimators, 
            max_depth=max_depth,
            criterion=criterion,
            class_weight={1:0.482,0:0.518},
            )
        
        return classifier
    
    def obj_func(self, trial: optuna.trial):
        if self.model_name == 'SVC':
            return ObjectiveFunctions.obj_func_svc(trial)
        elif self.model_name == 'RandomForest':
            return ObjectiveFunctions.obj_func_random_forest(trial)
        elif self.model_name == 'ExtraTreesClassifier':
            return ObjectiveFunctions.obj_func_extra_tree(trial)

In [14]:
# Define an empty dictionary to store the f1 scores for each feature type
f1_scores = {}

print(X_data.keys())
# Loop through each feature type in the X_data dictionary
for feature_type in X_data.keys():
    
    # Get the selected model for this feature type
    model = feature_model_map[feature_type]
    
    # Print the current model and feature type being trained
    print(f"Feature Type: {feature_type} | Training Model: {model}")
    
    # Shuffle the data and apply the data pipeline
    X, y = shuffle(X_data[feature_type], train_data.targets, random_state=config_map["random_seed"])
    data_pipeline = data_pipelines[feature_type]
    X = data_pipeline.fit_transform(X, y)
    
    # Define the objective function for the Optuna optimization
    def objective(trial):
        clf = ObjectiveFunctions(model).obj_func(trial)
        y_pred = cross_val_predict(clf, X, y, cv=5)
        result_values = evaluate_classification(
            y_pred, y, class_names=["Not Druggable", "Druggable"],
            save_outputs=None
        )
        return result_values["f1"]
    
    # Create an Optuna study and optimize the objective function
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100)
    
    # Print the best hyperparameters and f1 score obtained
    print('Best f1 Score: {}'.format(study.best_trial.value))
    print("Best Hyperparameters: {}".format(study.best_trial.params))
    
    # Store the best f1 score in the f1_scores dictionary
    f1_scores[feature_type] = study.best_trial.value
    
    # Save the best hyperparameters, study object, data pipeline object, and trained model
    params = study.best_trial.params
    model_dir = os.path.join(config_map["model_save_location"], feature_type, model)
    os.makedirs(model_dir, exist_ok=True)
    
    with open(os.path.join(model_dir, "hyperparams.json"), "w") as f:
        json.dump(params, f)

    classifier = None  
    if model == 'SVC':
        classifier = SVC(**params)
    elif model == 'RandomForest':
        classifier = RandomForestClassifier(**params)
    elif model == 'ExtraTreesClassifier':
        classifier = ExtraTreesClassifier(**params)
        
    classifier.fit(X, y)
    joblib.dump(classifier, os.path.join(model_dir, "model.sav"))
    joblib.dump(study, os.path.join(model_dir, "optuna_study.sav"))
    joblib.dump(data_pipeline, os.path.join(model_dir, "pipeline.sav"))
    
    # Print newlines to separate the outputs for each feature type
    print("\n\n")



[I 2023-05-11 12:20:54,247] A new study created in memory with name: no-name-cdf91df4-8cbf-43e0-9bc7-963cd8e7e4f5


dict_keys(['AAC', 'DPC', 'RScharge', 'RSDHP', 'RSpolar'])
Feature Type: AAC | Training Model: SVC


[I 2023-05-11 12:20:55,889] Trial 0 finished with value: 0.3416213416213416 and parameters: {'C': 3.2025116153174694e-06}. Best is trial 0 with value: 0.3416213416213416.


Overall accuracy: 0.519



[I 2023-05-11 12:20:56,516] Trial 1 finished with value: 0.8927256422429709 and parameters: {'C': 3.4189524993965006}. Best is trial 1 with value: 0.8927256422429709.


...

Overall accuracy: 0.884



[I 2023-05-11 13:01:09,239] Trial 98 finished with value: 0.8880896265169089 and parameters: {'n_estimators': 233, 'max_depth': 40, 'criterion': 'gini'}. Best is trial 53 with value: 0.8927968960863698.


Overall accuracy: 0.888



[I 2023-05-11 13:01:14,639] Trial 99 finished with value: 0.8892609120639845 and parameters: {'n_estimators': 271, 'max_depth': 51, 'criterion': 'log_loss'}. Best is trial 53 with value: 0.8927968960863698.


Overall accuracy: 0.889

Best f1 Score: 0.8927968960863698
Best Hyperparameters: {'n_estimators': 253, 'max_depth': 53, 'criterion': 'gini'}



